# JSON


In [ ]:
INSERT INTO PEOPLE
WITH base AS (
    SELECT 
        DATA:id::number as ID,
        DATA:timestamp::timestamp_ntz as DT,
        a.VALUE:value1::string as SUBSCRIBER_INFO,
		a.VALUE:type::string as SUBS_TYPE,
        a.VALUE:value2::string as SUBSCRIBER_DATA
      
    FROM DB.PEOPLE_SHAREJSON
        
  
    ,LATERAL FLATTEN(INPUT=>DATA:data) a
),
SUBSCRIBER_DATA as (
    SELECT 
        ID,
        FORM_ID,
        COALESCE(udf_initcap(Trim(PARSE_JSON("'name'"):first::string)),'NA') as FIRSTNAME,
        'NA' AS MIDDLENAME,
        COALESCE(udf_initcap(Trim(PARSE_JSON("'name'"):last::string)),'NA') as LASTNAME,
        COALESCE(lower(Trim("'email'")),'NA') AS EMAIL,
       
        FROM 
            (
                SELECT 
                    ID,
                    FORM_ID,
                    SUBSCRIBER_DATA, 
                    SUBS_TYPE, 
                    DT AS SUBSCRIBE_DT
                   
                FROM 
                    base 
                WHERE SUBS_TYPE in ('name','email')
                
            )
        PIVOT (MAX(SUBSCRIBER_DATA) FOR SUBS_TYPE IN 
                (
                SELECT 
                    b.VALUE:type::string AS OUTPUT  
                FROM 
                    DB.PEOPLE_SHAREJSON
                ,LATERAL FLATTEN(INPUT=> DATA:data) b
                WHERE trim(lower(b.VALUE:type::string)) IN ('name','email') 
                )
            )
        
        QUALIFY ROW_NUMBER() OVER(PARTITION BY ID,FORM_ID,ACCOUNT_ID ORDER BY SUBSCRIBE_DT) = 1